# Jupyter Notebook Instructions

This notebook is designed to train a model using a specified dataset. Please follow the instructions below to ensure the notebook runs correctly and efficiently.

## Instructions

1. **Run All Cells One by One**: Execute each cell in the notebook sequentially. Do not skip any cells to ensure all necessary components are initialized and loaded correctly.

2. **Training Cell**: The last cell in the notebook is the training cell. This cell initiates the model training process using the specified dataset.

3. **Change Dataset Directory**: You can change the directory of the dataset in the last cell. Ensure the correct path to your dataset is provided before running the training cell. 

4. **Optimal Environment**: For best performance, it is recommended to run this notebook on Kaggle with GPU T4x2.

5. **Run First Cell Twice**: On Kaggle, it is common that the first cell may not execute correctly on the first attempt. If this happens, simply run the first cell again to proceed.

## Notes

- Ensure you have all necessary dependencies installed and your environment is properly configured.
- Monitor the outputs and logs of each cell to catch any errors or issues early.
- Save your work frequently to avoid data loss.

Happy Training!

## Imports and Installs

In [2]:
!pip install -q git+https://github.com/thunlp/OpenDelta.git  

In [3]:
!pip install -q diffusers

In [4]:
!pip install --upgrade torch torchvision transformers peft 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 835.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:0000:0100:05m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 8.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:0000:01m00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━

In [5]:
!pip install -q pillow bigmodelvis pip install git+https://github.com/huggingface/transformers

In [6]:
import os, gc, requests
from io import BytesIO
import logging

# Suppress specific CUDA plugin registration errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# Core Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from opendelta import AdapterModel

# Visualization Libraries
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, HTML
from base64 import b64encode

# Helper Libraries
from bigmodelvis import Visualization
from torchvision import transforms as tfms
import torchvision.models as models
from tqdm.auto import tqdm


# Transformers Libraries
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    GPT2Tokenizer,
    GPT2Model,
    GPT2LMHeadModel,
    BlipProcessor,
    BlipForConditionalGeneration,
    CLIPTextModel,
    CLIPTokenizer
)

# Diffusers Libraries
from diffusers import (
    StableDiffusionPipeline,
    DiffusionPipeline,
    AutoencoderKL,
    UNet2DConditionModel,
    LMSDiscreteScheduler
)
from diffusers.models.modeling_outputs import Transformer2DModelOutput  # Fix for deprecated import

# Disable logging warnings
logging.disable(logging.WARNING)

# Matplotlib inline for Jupyter notebooks
%matplotlib inline

# CUDA settings
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
2024-06-20 18:48:32.744377: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 18:48:32.744508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 18:48:32.873772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning:

In [7]:
## Utils for Stable Diffusion
def load_image(p):
    '''
    Function to load images from a defined path
    '''
    return Image.open(p).convert('RGB').resize((512,512))
def pil_to_latents(image, vae,  device = "cuda"):
    '''
    Function to convert image to latents
    '''
    init_image = tfms.ToTensor()(image).unsqueeze(0) * 2.0 - 1.0
    init_image = init_image.to(device=device, dtype=torch.float16)
    init_latent_dist = vae.encode(init_image).latent_dist.sample() * 0.18215
    return init_latent_dist

def latents_to_tensor_image(latents, vae) :
    latents = (1 / 0.18215) * latents
    image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.permute(0, 2, 3, 1)
    images = (image * 255).round()
    return images

def latents_to_pil(latents, vae):
    '''
    Function to convert latents to images
    '''
    latents = (1 / 0.18215) * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images
def text_enc(prompts, tokenizer, text_encoder, maxlen=None, device = "cuda"):
    '''
    A function to take a texual promt and convert it into embeddings
    '''
    if maxlen is None: maxlen = tokenizer.model_max_length
    inp = tokenizer(prompts, padding="max_length", max_length=maxlen, truncation=True, return_tensors="pt")
    return text_encoder(inp.input_ids.to(device))[0].half()

In [8]:
# Define transformations
transform = tfms.Compose([
    tfms.Resize((256, 256)),
    tfms.ToTensor(),
    tfms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomDataLoader(Dataset):
    def __init__(self, image_dir, transform=transform):
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image
    
def tensor_to_pil(image_tensor):
    unloader = tfms.ToPILImage()
    image = image_tensor.cpu().clone()  # clone the tensor to not modify the original one
    image = image.squeeze(0)  # remove the batch dimension
    image = unloader(image)
    return image



In [9]:
class PerceptualLoss(nn.Module):
    def __init__(self, cnn, feature_layers):
        super(PerceptualLoss, self).__init__()
        self.cnn = cnn
        self.feature_layers = feature_layers
        self.criterion = nn.MSELoss()

    def forward(self, input, target):
        input_features = self.extract_features(input)
        target_features = self.extract_features(target)
        loss = 0
        for inp_feat, tgt_feat in zip(input_features, target_features):
            loss += self.criterion(inp_feat, tgt_feat)
        return loss

    def extract_features(self, x):
        features = []
        for name, layer in self.cnn._modules.items():
            x = layer(x)
            if name in self.feature_layers:
                features.append(x)
        return features


vgg = models.vgg16(pretrained=True).features
feature_layers = ['1', '3', '6', '8']  
perceptual_loss = PerceptualLoss(cnn=vgg, feature_layers=feature_layers).to('cuda')

transform_loss_input = tfms.Compose([
    tfms.Resize((224, 224)),
    tfms.ToTensor(),
    tfms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


class PermuteTransform:
    def __call__(self, tensor):
        return tensor.permute(0, 3, 1, 2)

# Define the image transformation pipeline
transform_loss_output = tfms.Compose([
    PermuteTransform(),  
    tfms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 201MB/s]  


In [10]:
class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith(('png', 'jpg', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        original_image = image.copy()  # Make a copy of the original image
        if self.transform:
            image = self.transform(image)
        return original_image, image


def train_model(model, dataloader, optimizer, num_epochs=5):
    model.train()  # Set the model to training mode
    mse_loss = nn.MSELoss()

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for images in dataloader:
            # Move images to the CPU
            images = images.to('cpu')

            # Generate latents from images
            image = tensor_to_pil(images[0])
            latent_input = model.image_to_latent(image)
            latent_input = F.interpolate(latent_input, size=(16, 16), mode='bilinear', align_corners=False)

            # Forward pass through the combined model
            latent_output = model(image)

            # Compute loss
            loss = mse_loss(latent_input, latent_output)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader)}')


## Combined model

In [11]:
class ImageToTextModel(nn.Module):
    def __init__(self, device="cpu"):
        super(ImageToTextModel, self).__init__()
        image_model_id = "Salesforce/blip-image-captioning-base"
        self.device = torch.device(device)
        self.model = BlipForConditionalGeneration.from_pretrained(image_model_id)
        self.processor = BlipProcessor.from_pretrained(image_model_id)

        self.model = self.model.to(device=self.device, dtype=torch.bfloat16)
        self.model.eval()

    def forward(self, image):
        inputs = self.processor(image, return_tensors="pt").to(self.device)
        out = self.model.generate(**inputs, max_new_tokens=200)

        caption = self.processor.decode(out[0], skip_special_tokens=True)

        return caption

    def save_model(self, model_path, processor_path):
        # Save the model state_dict
        torch.save(self.model.state_dict(), model_path)
        # Save the processor
        self.processor.save_pretrained(processor_path)

    def load_model(self, model_path, processor_path):
        # Load the model state_dict
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        # Load the processor
        self.processor = BlipProcessor.from_pretrained(processor_path)
        self.model = self.model.to(device=self.device, dtype=torch.bfloat16)
        self.model.eval()

In [12]:
class TextToTextModel(nn.Module):
    def __init__(self, device="cpu"):
        super(TextToTextModel, self).__init__()
        self.tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = GPT2LMHeadModel.from_pretrained('distilgpt2')
        self.device = device
        self.model = self.model.to(device=self.device, dtype=torch.bfloat16)
        

        # Apply delta tuning using OpenDelta
        self.delta_model = AdapterModel(backbone_model=self.model)
        self.delta_model.freeze_module(exclude=["deltas"], set_state_dict=True)

    def forward(self, inputs):
        # Improve the input prompt
        input_ids = inputs["input_ids"].clone().detach()
        attention_mask = inputs["attention_mask"].clone().detach()

        logits = self.model(input_ids=input_ids, attention_mask=attention_mask)["logits"]
        log_softmax_output = F.log_softmax(logits, dim=-1)

        return log_softmax_output.half()

    def logits_to_text(self, logits):
        text = self.tokenizer.decode(logits.argmax(dim=-1)[0], skip_special_tokens=True)
        return text
    
    
    def generate_best_text(self, inputs, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, \
                           top_p=0.95, temperature=1.0, do_sample=True) :
        
        output = self.model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            no_repeat_ngram_size=no_repeat_ngram_size,
            top_k=top_k,
            top_p=top_p,
            temperature=temperature,
            do_sample=do_sample
        )
        generated_text = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return generated_text

In [13]:
class TextToImageModel(nn.Module):
    def __init__(self, device="cuda"):
        super(TextToImageModel, self).__init__()
        self.tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
        self.text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to("cuda")
        self.vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", torch_dtype=torch.float16).to("cuda")
        self.scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)
        self.scheduler.set_timesteps(50)
        self.unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", torch_dtype=torch.float16).to("cuda")
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        
        self.unet.enable_gradient_checkpointing()


    def forward(self, text_encod, g=7.5, seed=100, steps=50, dim=128):
      # text = text_enc([prompt], maxlen=None)

        torch.manual_seed(seed)
        bs = 1
        uncond =  text_enc([" "] * bs, self.tokenizer, self.text_encoder, maxlen = text_encod.shape[1])
        emb = torch.cat([uncond, text_encod])

        latents = torch.randn((bs, self.unet.config.in_channels, dim//8, dim//8))
        self.scheduler.set_timesteps(steps)
        latents = latents.to("cuda").half() * self.scheduler.init_noise_sigma

        for i,ts in enumerate(tqdm(self.scheduler.timesteps)):
          # We need to scale the i/p latents to match the variance
          inp = self.scheduler.scale_model_input(torch.cat([latents] * 2), ts)
          # Predicting noise residual using U-Net
          u, t = self.unet(inp, ts.long().to("cuda"), encoder_hidden_states=emb).sample.chunk(2)
          pred = u + g*(t-u)
          latents = self.scheduler.step(pred, ts, latents).prev_sample
        return latents

    def latent_to_image(self, latents) :
        images = latents_to_pil(latents, self.vae)
        for img in images:display(img)
            
    def latent_to_tensor(self, latents):
        return latents_to_tensor_image(latents, self.vae)

    def image_to_latent(self, image) :
        image = image
        return pil_to_latents(image, self.vae)

    def text_to_text_encod(self, text) :
        text = self.text_enc([text], self.tokenizer, self.text_encoder, maxlen=None)
        



In [14]:

class CombinedModel(nn.Module):
    def __init__(self, device_i2t="cpu", device_t2t="cuda", device_t2i="cuda", hidden_dim=768, voc_GPT2_size=50257, target_seq_len=77, print_intermediate=True, \
                 min_transition = -50, max_transition = 50, text_to_image_steps = 50):
        super(CombinedModel, self).__init__()
        self.device_i2t, self.device_t2t, self.device_t2i = device_i2t, device_t2t, device_t2i
        self.i2t = ImageToTextModel(self.device_i2t)
        self.t2t = TextToTextModel(self.device_t2t)
        self.t2i = TextToImageModel(self.device_t2i)
        self.hidden_dim = hidden_dim
        self.voc_GPT2_size = voc_GPT2_size
        self.target_seq_len = target_seq_len
        self.print_intermediate = print_intermediate
        self.text_to_image_steps = text_to_image_steps
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, image_pil, dim_output = 128):
        text_description = self.i2t(image_pil)
        if self.print_intermediate : print(f"Text from image: {text_description}")
            
        improved_prompt = f"Do a short image description:{text_description}"
        inputs = self.t2t.tokenizer(improved_prompt, return_tensors="pt", padding=True, truncation=True)
        inputs = inputs.to(self.t2t.model.device)
        
        logits = self.t2t(inputs)
        if self.print_intermediate : print(f"Refined text: {self.t2t.generate_best_text(inputs)}")
        
        
        ################## Transition logits to input_ids ################## 
        projection_layer = nn.Linear(self.voc_GPT2_size, self.hidden_dim).half().to("cuda")  # Change the data type of the linear layer to half (float16)
        projected_logits = projection_layer(logits)  # [batch_size, seq_len1, hidden_dim]
        projected_logits = projected_logits.permute(0, 2, 1)  # [batch_size, hidden_dim, seq_len1]
        interpolated_logits = F.interpolate(projected_logits, size=self.target_seq_len, mode='linear')
        interpolated_logits = interpolated_logits.permute(0, 2, 1)  # [batch_size, target_seq_len, hidden_dim]
        if self.training:
            interpolated_logits = self.dropout(interpolated_logits)
        
        ################## ################## ################## 
        latents = self.t2i(interpolated_logits.to("cuda"), steps = self.text_to_image_steps, dim = dim_output)
        image_tensor = self.latent_to_tensor(latents)
        
        return image_tensor
    
    def latent_to_image(self, latents) :
        return self.t2i.latent_to_image(latents)
    def latent_to_tensor(self, latents) :
        return self.t2i.latent_to_tensor(latents)
    def image_to_latent(self, image_pil) :
        return self.t2i.image_to_latent(image_pil)
    
    def freeze_all_layers(self) :
        for model in [self.i2t, self.t2t, self.t2i] :
            for param in model.parameters():
                param.requires_grad = False
    
    def unfreeze_layers(self, model_index = 1, list_name_to_unfreeze = [["0", "mlp"], ["adapter"]]) :
        for name, param in combinedModel.t2t.named_parameters():
            for sub_list in list_name_to_unfreeze :
                if all(element in name for element in sub_list) :
                    param.requires_grad = True


In [23]:
def pil_to_tensor(image, size=(224, 224)):
    # Resize the image
    image = image.resize(size, Image.BILINEAR)
    # Convert the image to a NumPy array
    image_array = np.array(image, dtype=np.float32)
    # Convert the NumPy array to a PyTorch tensor
    image_tensor = torch.from_numpy(image_array)
    return image_tensor.unsqueeze(0).to("cuda")


# Function to load images from a directory
def load_images_from_directory(directory, num_images):
    images = []
    for i, filename in enumerate(os.listdir(directory)):
        if i >= num_images:
            break
        img_path = os.path.join(directory, filename)
        image = Image.open(img_path).convert('RGB')
        images.append(image)
    return images

In [24]:
gc.collect()
torch.cuda.empty_cache()

# Constants
NUM_EPOCHS = 100
THRESHOLD = 1e-6  # Define a threshold to detect if the loss is stuck

# Directories and transformations
image_dir = '/kaggle/input/animals10/raw-img/elefante'

# Load first five images
images = load_images_from_directory(image_dir, num_images = 3)

# Initialize model
combinedModel = CombinedModel(text_to_image_steps=30)
combinedModel.freeze_all_layers()
combinedModel.unfreeze_layers(list_name_to_unfreeze=["adapter"])

# Set model to training mode
combinedModel.train()

# Initialize optimizer
optimizer = optim.Adam(combinedModel.parameters(), lr=1e-4)

# Initialize previous loss to a very high value
previous_loss = float('inf')
use_perceptual_loss = False
mse_loss = nn.MSELoss()

# Training loop
for epoch in range(NUM_EPOCHS):
    for image in images :
        image_tensor_input = transform_loss_input(image).unsqueeze(0).to('cuda')
        latent_input = combinedModel.image_to_latent(image)
        latent_input = F.interpolate(latent_input, size=(16, 16), mode='bilinear', align_corners=False)
        tensor_image = combinedModel(image, dim_output=224)

        image_tensor_output = transform_loss_output(tensor_image).to(torch.float32).to("cuda")

        # Compute loss
        if use_perceptual_loss:
            x = tfms.ToPILImage()(tensor_image.squeeze(0).permute(2, 0, 1))
            image_tensor_output = transform_loss_input(x).unsqueeze(0).to('cuda')
            loss = perceptual_loss(image_tensor_input, image_tensor_output)
        else:
            loss = mse_loss(pil_to_tensor(image), tensor_image.to(torch.float32))

        # Check if loss is stuck
        if abs(loss.item() - previous_loss) < THRESHOLD:
            use_perceptual_loss = False
        else:
            use_perceptual_loss = True

        # Update previous loss
        previous_loss = loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(combinedModel.parameters(), max_norm=1.0)
        optimizer.step()

        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item()}')

# Set model to evaluation mode after training if needed
combinedModel.eval()

Text from image: a baby elephant and its mother
Refined text: Do a short image description:a baby elephant and its mother that made it into a new species of fruit.

A video has been created of its own so far.The original image was originally shown at a meeting of an audience at the Natural


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch [1/100], Loss: 8580.8994140625
Text from image: a man is touching an elephant in a zoo
Refined text: Do a short image description:a man is touching an elephant in a zoo in Uganda, and that is why it is so important. When you look at a picture of a donkey, you see that he has to be put on a bench, or


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch [1/100], Loss: 96.25025939941406
Text from image: two elephants walking together
Refined text: Do a short image description:two elephants walking together in between, and to see that it is true, which gives us some details about the evolution of a species (or in other cases, such as how it evolved, or if an animal evolved).


...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch [2/100], Loss: 132.69699096679688
Text from image: a baby elephant and its mother
Refined text: Do a short image description:a baby elephant and its mother in a tank, to which I think it is a great example of the world of elephants.


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch [3/100], Loss: 104.0108871459961


KeyboardInterrupt: 